# Image Denoising using Fourier Low-Pass Filtering

This notebook demonstrates how to denoise images using Fourier Transform and Low-Pass Filtering.
We'll also calculate various quality metrics to evaluate the denoising performance.

In [ ]:
# Import all the necessary libraries
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from scipy.stats import entropy as scipy_entropy
import os

# Set up matplotlib for better visualization
plt.rcParams['figure.figsize'] = (15, 10)
print("All libraries imported successfully!")

In [ ]:
# Function to create a low-pass filter in the frequency domain
def create_lowpass_filter(shape, cutoff_frequency):
    """
    Creates a circular low-pass filter mask in the frequency domain.
    
    Parameters:
    - shape: tuple of (height, width) of the image
    - cutoff_frequency: radius of the circular filter (lower = more smoothing)
    
    Returns:
    - filter_mask: 2D numpy array with 1s inside the circle and 0s outside
    """
    rows, cols = shape
    
    # Find the center of the frequency domain (where DC component is located)
    center_row, center_col = rows // 2, cols // 2
    
    # Create a meshgrid to calculate distances from center
    y, x = np.ogrid[:rows, :cols]
    
    # Calculate the distance of each point from the center
    distance_from_center = np.sqrt((x - center_col)**2 + (y - center_row)**2)
    
    # Create the mask: 1 if within cutoff radius, 0 otherwise
    filter_mask = distance_from_center <= cutoff_frequency
    
    return filter_mask.astype(float)

In [ ]:
# Function to apply Fourier Low-Pass Filter to denoise an image
def fourier_lowpass_filter(image, cutoff_frequency):
    """
    Applies a low-pass filter in the frequency domain to remove high-frequency noise.
    
    Parameters:
    - image: input grayscale image (2D numpy array)
    - cutoff_frequency: radius of the low-pass filter
    
    Returns:
    - denoised_image: filtered image in spatial domain
    - fft_shifted: shifted FFT of original image (for visualization)
    - filtered_fft: filtered FFT (for visualization)
    """
    # Step 1: Apply 2D Fast Fourier Transform to convert image to frequency domain
    fft = np.fft.fft2(image)
    
    # Step 2: Shift the zero-frequency component to the center
    # This makes visualization and filtering easier
    fft_shifted = np.fft.fftshift(fft)
    
    # Step 3: Create the low-pass filter mask
    filter_mask = create_lowpass_filter(image.shape, cutoff_frequency)
    
    # Step 4: Apply the filter by multiplying in frequency domain
    # This removes high-frequency components (which often represent noise)
    filtered_fft = fft_shifted * filter_mask
    
    # Step 5: Shift back the zero-frequency component
    fft_ishifted = np.fft.ifftshift(filtered_fft)
    
    # Step 6: Apply Inverse FFT to get back to spatial domain
    denoised_image = np.fft.ifft2(fft_ishifted)
    
    # Step 7: Take the real part and convert to proper range
    # (imaginary part should be negligible due to numerical errors)
    denoised_image = np.real(denoised_image)
    
    # Clip values to valid range [0, 255]
    denoised_image = np.clip(denoised_image, 0, 255).astype(np.uint8)
    
    return denoised_image, fft_shifted, filtered_fft

In [ ]:
# Function to calculate image entropy
def calculate_entropy(image):
    """
    Calculates the entropy of an image, which measures randomness/information content.
    Higher entropy = more random/noisy, Lower entropy = more uniform/smooth
    
    Parameters:
    - image: input image (2D numpy array)
    
    Returns:
    - entropy_value: entropy of the image
    """
    # Calculate histogram to get probability distribution
    histogram, _ = np.histogram(image.flatten(), bins=256, range=(0, 256), density=True)
    
    # Remove zero probabilities to avoid log(0)
    histogram = histogram[histogram > 0]
    
    # Calculate entropy using Shannon's formula: -sum(p * log2(p))
    entropy_value = scipy_entropy(histogram, base=2)
    
    return entropy_value

In [ ]:
# Function to calculate noise statistics between two images
def calculate_noise_statistics(original, denoised):
    """
    Calculates noise variance and standard deviation between original and denoised images.
    These metrics help us understand how much noise was removed.
    
    Parameters:
    - original: original noisy image
    - denoised: denoised image
    
    Returns:
    - noise_variance: variance of the noise (difference between images)
    - noise_std: standard deviation of the noise
    """
    # Calculate the difference (residual noise)
    noise = original.astype(float) - denoised.astype(float)
    
    # Calculate variance: average of squared deviations from mean
    noise_variance = np.var(noise)
    
    # Calculate standard deviation: square root of variance
    noise_std = np.std(noise)
    
    return noise_variance, noise_std

In [ ]:
# Function to calculate all quality metrics
def calculate_quality_metrics(original, noisy, denoised):
    """
    Calculates comprehensive quality metrics to evaluate denoising performance.
    
    Parameters:
    - original: original clean image (if available)
    - noisy: noisy input image
    - denoised: denoised output image
    
    Returns:
    - metrics_dict: dictionary containing all calculated metrics
    """
    metrics = {}
    
    # SSIM: Structural Similarity Index (ranges from -1 to 1, 1 is perfect)
    # Measures perceived quality based on luminance, contrast, and structure
    if original is not None:
        metrics['SSIM (Original vs Denoised)'] = ssim(original, denoised, data_range=255)
    metrics['SSIM (Noisy vs Denoised)'] = ssim(noisy, denoised, data_range=255)
    
    # PSNR: Peak Signal-to-Noise Ratio (in dB, higher is better)
    # Measures the ratio between maximum signal power and noise power
    if original is not None:
        metrics['PSNR (Original vs Denoised)'] = psnr(original, denoised, data_range=255)
    metrics['PSNR (Noisy vs Denoised)'] = psnr(noisy, denoised, data_range=255)
    
    # Entropy: Measure of randomness (lower after denoising is usually better)
    metrics['Entropy (Noisy)'] = calculate_entropy(noisy)
    metrics['Entropy (Denoised)'] = calculate_entropy(denoised)
    if original is not None:
        metrics['Entropy (Original)'] = calculate_entropy(original)
    
    # Noise Statistics: Variance and Standard Deviation of removed noise
    noise_var, noise_std = calculate_noise_statistics(noisy, denoised)
    metrics['Noise Variance'] = noise_var
    metrics['Noise Std Dev'] = noise_std
    
    return metrics

In [ ]:
# Function to display results with visualizations
def display_results(original, noisy, denoised, fft_shifted, filtered_fft, metrics):
    """
    Displays a comprehensive visualization of the denoising process and results.
    """
    # Create a figure with multiple subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # Row 1: Show images
    if original is not None:
        axes[0, 0].imshow(original, cmap='gray')
        axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
        axes[0, 0].axis('off')
    else:
        axes[0, 0].text(0.5, 0.5, 'Original Not Available', 
                       ha='center', va='center', fontsize=12)
        axes[0, 0].axis('off')
    
    axes[0, 1].imshow(noisy, cmap='gray')
    axes[0, 1].set_title('Noisy Image (Input)', fontsize=14, fontweight='bold')
    axes[0, 1].axis('off')
    
    axes[0, 2].imshow(denoised, cmap='gray')
    axes[0, 2].set_title('Denoised Image (Output)', fontsize=14, fontweight='bold')
    axes[0, 2].axis('off')
    
    # Row 2: Show frequency domain representations
    # Display magnitude spectrum (log scale for better visualization)
    magnitude_spectrum = np.log(np.abs(fft_shifted) + 1)
    axes[1, 0].imshow(magnitude_spectrum, cmap='gray')
    axes[1, 0].set_title('FFT Magnitude Spectrum\n(Before Filtering)', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')
    
    # Display filtered magnitude spectrum
    filtered_magnitude = np.log(np.abs(filtered_fft) + 1)
    axes[1, 1].imshow(filtered_magnitude, cmap='gray')
    axes[1, 1].set_title('FFT Magnitude Spectrum\n(After Low-Pass Filtering)', fontsize=12, fontweight='bold')
    axes[1, 1].axis('off')
    
    # Display noise removed (difference image)
    noise_removed = np.abs(noisy.astype(float) - denoised.astype(float))
    axes[1, 2].imshow(noise_removed, cmap='hot')
    axes[1, 2].set_title('Removed Noise\n(Difference)', fontsize=12, fontweight='bold')
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print quality metrics in a formatted way
    print("\n" + "="*60)
    print("QUALITY METRICS")
    print("="*60)
    for metric_name, value in metrics.items():
        print(f"{metric_name:.<40} {value:.4f}")
    print("="*60)

## Main Processing Section

Now let's process your images! You'll need to provide:
1. Path to the noisy image
2. Path where you want to save the denoised image
3. (Optional) Path to the original clean image for comparison

In [ ]:
# ========== USER INPUT SECTION ==========
# Replace these paths with your actual file locations

# Path to your noisy image (the image you want to denoise)
noisy_image_path = input("Enter the path to the noisy image: ")

# Path where you want to save the denoised image
output_image_path = input("Enter the path to save the denoised image: ")

# Optional: Path to original clean image (for comparison metrics)
# Leave empty if you don't have the original
original_image_path = input("Enter the path to the original image (press Enter to skip): ")
if original_image_path.strip() == "":
    original_image_path = None

# Cutoff frequency for the low-pass filter
# Lower values = more smoothing (removes more noise but may blur)
# Higher values = less smoothing (preserves details but may keep noise)
# Typical range: 30-100 depending on image size and noise level
cutoff_frequency = 50

print("\nSettings configured successfully!")

In [ ]:
# ========== IMAGE LOADING ==========
print("Loading images...")

# Load the noisy image
noisy_image = cv2.imread(noisy_image_path, cv2.IMREAD_GRAYSCALE)
if noisy_image is None:
    raise ValueError(f"Could not load image from {noisy_image_path}. Please check the path.")

# Load the original image if provided
original_image = None
if original_image_path:
    original_image = cv2.imread(original_image_path, cv2.IMREAD_GRAYSCALE)
    if original_image is None:
        print(f"Warning: Could not load original image from {original_image_path}")
        original_image = None
    else:
        # Make sure dimensions match
        if original_image.shape != noisy_image.shape:
            print("Warning: Original and noisy images have different dimensions!")
            original_image = cv2.resize(original_image, (noisy_image.shape[1], noisy_image.shape[0]))

print(f"Noisy image loaded: {noisy_image.shape}")
if original_image is not None:
    print(f"Original image loaded: {original_image.shape}")

In [ ]:
# ========== APPLY DENOISING ==========
print(f"\nApplying Fourier Low-Pass Filter with cutoff frequency: {cutoff_frequency}...")

# Apply the denoising algorithm
denoised_image, fft_shifted, filtered_fft = fourier_lowpass_filter(noisy_image, cutoff_frequency)

print("Denoising complete!")

In [ ]:
# ========== CALCULATE METRICS ==========
print("\nCalculating quality metrics...")

# Calculate all quality metrics
metrics = calculate_quality_metrics(original_image, noisy_image, denoised_image)

print("Metrics calculated!")

In [ ]:
# ========== DISPLAY RESULTS ==========
# Show comprehensive visualization
display_results(original_image, noisy_image, denoised_image, fft_shifted, filtered_fft, metrics)

In [ ]:
# ========== SAVE DENOISED IMAGE ==========
print(f"\nSaving denoised image to: {output_image_path}")

# Create output directory if it doesn't exist
output_dir = os.path.dirname(output_image_path)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the denoised image
cv2.imwrite(output_image_path, denoised_image)

print("✓ Denoised image saved successfully!")

## Understanding the Metrics

**SSIM (Structural Similarity Index):**
- Range: -1 to 1 (1 is perfect similarity)
- Measures structural similarity between images
- Higher is better

**PSNR (Peak Signal-to-Noise Ratio):**
- Measured in decibels (dB)
- Typical values: 20-50 dB
- Higher is better (less distortion)

**Entropy:**
- Measures randomness/information content
- Lower after denoising usually indicates noise reduction

**Noise Variance & Std Dev:**
- Measures the amount of noise removed
- Shows the magnitude of changes made during denoising

## Tips for Better Results

1. **Adjust Cutoff Frequency:** If the image is too blurry, increase it. If noise remains, decrease it.
2. **Image Type:** This method works best for images with Gaussian noise.
3. **Color Images:** For color images, apply the filter to each channel separately.
4. **Compare Results:** Try different cutoff frequencies to find the optimal value for your image.